In [ ]:
# uncomment to install requirements
#!pip install tqdm pandas keras-tuner scikit-learn tensorflow matplotlib seaborn numpy ipykernel jupyter

In [ ]:
# Work in progress place to put the main code
# TODO next: reorganize into time windows by user and ordered by data ()
import tqdm
import pandas as pd
import numpy as np
import keras_tuner as kt
from tensorflow.keras.layers import Embedding, Dense, LSTM, GRU
from tensorflow.keras import Input, Model
from sklearn.model_selection import train_test_split
from random import choice, sample
import tensorflow as tf
from gc import collect
#import multiprocessing
from pickle import dump, load
#pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
#pool = multiprocessing.Pool(processes=6)

# try:
#   tf.keras.mixed_precision.set_global_policy('mixed_float16')
# except:
#   pass

In [ ]:
DATA_FILE_NAME = 'top_data_dimreduce_20.csv'
def load_data():
    """Takes one hour to run (single thread), so it is probably best to downlaod user.pkl

       File format is for local machine. Alter if running on colab
    """
    try:
        with open(r"users.pkl", "rb") as f:
            users = load(f)
    except:
        data = pd.read_csv(DATA_FILE_NAME, sep=',', header=0, names=['Movie','User','Rating','Date','Rating_uco','Rating_mco'])
        data['Date'] = pd.to_datetime(data['Date'])
        data['Date'] = data['Date'].astype('int64')/max(data['Date'].astype('int64'))
        data = data.dropna(axis=0)
        total_ratings = data.shape[0]
        print(f"Total number of ratings: {total_ratings}")
        data.sort_values(['User','Date'],inplace=True)
        data.head()
        users = {i:[] for i in set(data['User'])}
        sett = [[data.iloc[rating].User,[data.iloc[rating].Date, data.iloc[rating].Movie, data.iloc[rating].Rating_uco, data.iloc[rating].Rating_mco, data.iloc[rating].Rating]] for rating in range(total_ratings)]
        for [user, settt] in sett: users[user].append(settt)
        with open(r"users.pkl", "wb") as f:
            dump(users, f)
        del data
        del settt,sett
    return users

class rate_movies:
    def __init__(self):
        self.bs = 100
        self.lr = 0.01
        self.model = None
        self.data = None
        self.train_data_x = None
        #self.val_data_x = None
        self.test_data_x = None
        self.train_data_y = None
        #self.val_data_y = None
        self.test_data_y = None
        self.seq_length = 10
        self.buffer_size = 10000
        self.seed = 897
        self.model = None
        self.tuned_model = None
        self.tuner = None
        self.split_ratios = {'train':9/10,'test':1} # the iterative percentage of data to remove in sequence. Result: 70-20-10 split *Edit: val dataset is now split during fit
    
    def split_load_data(self):
        train_needed = False
        try: # try to load the data. Not stylistic method.
            with open(r"trainx.pkl", "rb") as f:
                self.train_data_x = load(f)
            #with open(r"valx.pkl", "rb") as f:
                #self.val_data_x = load(f)
            with open(r"testx.pkl", "rb") as f:
                self.test_data_x = load(f)
            with open(r"trainy.pkl", "rb") as f:
                self.train_data_y = load(f)
            #with open(r"valy.pkl", "rb") as f:
                #self.val_data_y = load(f)
            with open(r"testy.pkl", "rb") as f:
                self.test_data_y = load(f)

            #self.train_data_x, self.train_data_y, self.val_data_x, self.val_data_y, self.test_data_x, self.test_data_y  = tf.ragged.constant(self.train_data_x), tf.ragged.constant(self.train_data_y), tf.ragged.constant(self.val_data_x), tf.ragged.constant(self.val_data_y), tf.ragged.constant(self.test_data_x), tf.ragged.constant(self.test_data_y)

        except: # process the data and then create dump files if we cannot load
            train_needed = True

        if train_needed:
            self.data = load_data()
            selection = []
            i = 0 #keeps track of what split ratio to apply
            for ratio in tqdm.tqdm(self.split_ratios):
                selection.append(sample(sorted(self.data), k=int(len(self.data)*self.split_ratios[ratio]))) #random sample of the users (selection means all user data goes to that dataset)
                keys = selection[i]  # copy user numbers bc we are about to replace it with their data
                selection[i] = [self.data[x] for x in selection[i]] 
                i+=1  # next dataset
                for key in keys: # for every username in the sample, remove it from the selection pool
                    if key in self.data: del self.data[key]

            train_data, test_data = selection  # split list set into the datasets
            #del selection, self.data, i, keys # saves memory... remove if needed.

            def labels(input):  # probably working now
                # input = [[(input[:-1],[input[-1]]) for input in input]  for input in input]  # Needed: create window... last element of window must become label... convert to tenseors... batch
                i = 0
                c = len(input)
                h = 0
                x,y = [],[]

                for [date, movie, r, rr, rating] in input:
                    h+=1
                    if h == c:
                        x[-1]=x[-1]+[date, movie, r, rr]
                        y.append([rating])
                        #x[-1],y[-1] = tf.ragged.constant(x[-1]), tf.ragged.constant(y[-1])
                        i = 0
                    elif i == 0:
                        x.append([date, movie, r, rr, rating])
                        i+=1
                    elif i < self.seq_length:
                        x[-1]=x[-1]+[date, movie, r, rr, rating]
                        i += 1
                    else:
                        x[-1]=x[-1]+[date, movie, r, rr]
                        y.append([rating])
                        #x[-1],y[-1] = tf.ragged.constant(x[-1]), tf.ragged.constant(y[-1])
                        i = 0
                #dataset = zip(tf.data.Dataset.from_tensor_slices(x).batch(self.bs, drop_remainder=True), tf.data.Dataset.from_tensor_slices(y).batch(self.bs, drop_remainder=True))
                #del x,y,i,h,c
                return (x,y)

            def iterate(input):
                x = []
                y=[]
                for i in tqdm.tqdm(input):
                    ii,iii = labels(i)
                    x.append(ii)
                    y.append(iii)
                return x,y

            self.train_data_x, self.train_data_y = iterate(train_data)
            #self.val_data_x, self.val_data_y   = iterate(val_data)
            self.test_data_x, self.test_data_y  = iterate(test_data)
            self.train_data_x, self.train_data_y, self.test_data_x, self.test_data_y  = tf.ragged.constant(self.train_data_x).to_tensor(), tf.ragged.constant(self.train_data_y).to_tensor(), tf.ragged.constant(self.test_data_x).to_tensor(), tf.ragged.constant(self.test_data_y).to_tensor()

            with open(r"trainx.pkl", "wb") as f:
                dump(self.train_data_x, f)
            # with open(r"valx.pkl", "wb") as f:
            #     dump(self.val_data_x, f)
            with open(r"testx.pkl", "wb") as f:
                dump(self.test_data_x, f)
            with open(r"trainy.pkl", "wb") as f:
                dump(self.train_data_y, f)
            # with open(r"valy.pkl", "wb") as f:
            #     dump(self.val_data_y, f)
            with open(r"testy.pkl", "wb") as f:
                dump(self.test_data_y, f)


    def recc_model(self):
        def model_arch(layers=2, units=228, dropout_rate=0.2):
            inputs = tf.keras.Input(batch_input_shape=[self.bs, None, 58])
            x = tf.keras.layers.Normalization()(inputs)
            for _ in range(layers): 
                x = LSTM(units, return_sequences=True, stateful=True)(x)
            if dropout_rate > 0.0: x = tf.keras.layers.Dropout(dropout_rate)(x)
            x = Dense(1, activation='swish') (x)
            model = tf.keras.Model(inputs,x)
            loss= tf.keras.optimizers.Adam()
            model.compile(optimizer=loss, loss='mse',metrics=tf.keras.metrics.RootMeanSquaredError(name='root_mean_squared_error'))
            return model
        self.model = model_arch()
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)]
        self.model.fit(self.train_data_x,self.train_data_y, validation_split=0.3,epochs = 500, callbacks=callbacks,batch_size=self.bs)

    def keras_tuner(self,search): #In progress.. too many things rn to test so will eliminate some choices
        if search:
            def model_arch(hp):
                inputs = tf.keras.Input(batch_input_shape=[self.bs, None, 58])
                x = tf.keras.layers.Normalization()(inputs)
                for _ in range(hp.Int('layers',0,4)): 
                    x = LSTM(hp.Int('units',1,512), return_sequences=True, stateful=True)(x)
                if hp.Float('dropout_rate',0.0,.5) > 0.0: x = tf.keras.layers.Dropout(hp.Float('dropout_rate',0.0,.5))(x)
                x = Dense(1, activation=hp.Choice('final_activation',['swish','relu','selu','elu','gelu'])) (x)
                model = tf.keras.Model(inputs,x)
                loss= tf.keras.optimizers.Adam(learning_rate=hp.Float('lr',0,0.05))
                model.compile(optimizer=loss, loss='mse')#,metrics=tf.keras.metrics.RootMeanSquaredError(name='root_mean_squared_error')
                return model
        else: # return the best model (currently some settings are hard-coded to predicted best values because the search ran out of time)
            def model_arch():
                inputs = tf.keras.Input(batch_input_shape=[self.bs, None, 58])
                x = tf.keras.layers.Normalization()(inputs)
                for _ in range(4): 
                    x = LSTM(319, return_sequences=True, stateful=True)(x)
                x = tf.keras.layers.Dropout(0.39063)(x)
                x = Dense(1, activation='gelu') (x)
                model = tf.keras.Model(inputs,x)
                loss= tf.keras.optimizers.Adam(learning_rate=0.00097747)
                model.compile(optimizer=loss, loss='mse',metrics=tf.keras.metrics.RootMeanSquaredError(name='root_mean_squared_error'))#,metrics=tf.keras.metrics.RootMeanSquaredError(name='root_mean_squared_error')
                return model
        if search:
            self.tuner = kt.Hyperband(model_arch,objective='val_loss', max_epochs=500, hyperband_iterations=5)
            callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=1,restore_best_weights=True)]
            self.tuner.search(self.train_data_x,self.train_data_y, validation_split=0.3,epochs = 50, callbacks=callbacks,batch_size=self.bs)
            *best_parameters, = self.tuner.get_best_hyperparameters()[0]
            dump(best_parameters,open( "best_parameters.pkl", "wb" ))
        
        self.tuned_model = model_arch()
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)]
        #self.tuned_model.summary()
        self.tuned_model.fit(self.train_data_x,self.train_data_y, validation_split=0.3,epochs = 500, callbacks=callbacks,batch_size=self.bs)

        

    
    def eval(self,model):
        model.evaluate(self.test_data_x,self.test_data_y,batch_size=self.bs)
        pass


rec_system = rate_movies()

rec_system.split_load_data()
rec_system.train_data_y.shape

TensorShape([9000, 181, 1])

In [ ]:
rec_system.recc_model()

Epoch 1/500
63/63 [==============================] - 4s 47ms/step - loss: 0.8338 - root_mean_squared_error: 0.9131 - val_loss: 0.5782 - val_root_mean_squared_error: 0.7604
Epoch 2/500
63/63 [==============================] - 3s 41ms/step - loss: 0.5814 - root_mean_squared_error: 0.7625 - val_loss: 0.5645 - val_root_mean_squared_error: 0.7513
Epoch 3/500
63/63 [==============================] - 3s 40ms/step - loss: 0.5657 - root_mean_squared_error: 0.7521 - val_loss: 0.5459 - val_root_mean_squared_error: 0.7389
Epoch 4/500
63/63 [==============================] - 2s 40ms/step - loss: 0.5587 - root_mean_squared_error: 0.7475 - val_loss: 0.5444 - val_root_mean_squared_error: 0.7378
Epoch 5/500
63/63 [==============================] - 2s 38ms/step - loss: 0.5557 - root_mean_squared_error: 0.7454 - val_loss: 0.5418 - val_root_mean_squared_error: 0.7361
Epoch 6/500
63/63 [==============================] - 2s 39ms/step - loss: 0.5543 - root_mean_squared_error: 0.7445 - val_loss: 0.5427 - val_

In [ ]:
rec_system.eval(rec_system.model) # evaluate the model on the test set

10/10 [==============================] - 1s 13ms/step - loss: 0.5779 - root_mean_squared_error: 0.7602


In [ ]:
a,b = rec_system.model.predict(rec_system.test_data_x,batch_size=rec_system.bs),rec_system.test_data_y  # create prediction and label pairs

def add(x,y): return list([float(x)]) + list([float(y)]) # sort prediction and label pairs
*pred_pairs, = map(add,a[0],b[0])

pred_pairs = list(pred_pairs)

In [ ]:
pred_pairs[0:10]

[[3.2046353816986084, 5.0],
 [3.7242283821105957, 4.0],
 [3.743262767791748, 3.0],
 [3.4734957218170166, 5.0],
 [3.4189014434814453, 5.0],
 [3.506932258605957, 3.0],
 [3.4454028606414795, 3.0],
 [3.433408498764038, 5.0],
 [3.652507781982422, 5.0],
 [3.4825100898742676, 3.0]]

In [ ]:
rec_system.keras_tuner(search=True)

Trial 615 Complete [00h 00m 15s]
val_loss: 0.5495085120201111

Best val_loss So Far: 0.541039764881134
Total elapsed time: 00h 02m 43s

Search: Running Trial #616

Value             |Best Value So Far |Hyperparameter
3                 |4                 |layers
0.34145           |0.39063           |dropout_rate
selu              |gelu              |final_activation
0.070598          |0.00097747        |lr
348               |319               |units
19                |7                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/19
63/63 [==============================] - ETA: 0s - loss: 0.8753

In [ ]:
rec_system.keras_tuner(search=False) #

Epoch 1/500
63/63 [==============================] - 9s 100ms/step - loss: 1.1561 - root_mean_squared_error: 1.0752 - val_loss: 0.6028 - val_root_mean_squared_error: 0.7764
Epoch 2/500
63/63 [==============================] - 6s 90ms/step - loss: 0.6043 - root_mean_squared_error: 0.7774 - val_loss: 0.5609 - val_root_mean_squared_error: 0.7489
Epoch 3/500
63/63 [==============================] - 6s 91ms/step - loss: 0.5739 - root_mean_squared_error: 0.7576 - val_loss: 0.5489 - val_root_mean_squared_error: 0.7408
Epoch 4/500
63/63 [==============================] - 6s 91ms/step - loss: 0.5702 - root_mean_squared_error: 0.7551 - val_loss: 0.5489 - val_root_mean_squared_error: 0.7408
Epoch 5/500
63/63 [==============================] - 6s 92ms/step - loss: 0.5947 - root_mean_squared_error: 0.7712 - val_loss: 0.5443 - val_root_mean_squared_error: 0.7378
Epoch 6/500
63/63 [==============================] - 6s 95ms/step - loss: 0.5608 - root_mean_squared_error: 0.7489 - val_loss: 0.5416 - val